In [3]:
%pip install langchain_text_splitters

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_text_splitters-1.0.0-py3-none-any.whl.metadata (2.6 kB)
Using cached langchain_text_splitters-1.0.0-py3-none-any.whl (33 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.append('..')
from src.embeddings import EmbeddingsManager
from src.document_processor import DocumentProcessor
import json
print("Imports successful")

Imports successful


In [3]:
with open('../data/processed/chunks.json','r') as f:
    chunks=json.load(f)
    
print(f'Loaded {len(chunks)} chunks')
print(f"sample chunk:{chunks[0]['text']}")    

Loaded 13 chunks
sample chunk:═══════════════════════════════════════════════════════════
         HEALTHGUARD INSURANCE POLICY DOCUMENT
                    Policy Version 2.1
═══════════════════════════════════════════════════════════

SECTION 1: WAITING PERIODS AND IMMEDIATE COVERAGE
─────────────────────────────────────────────────────────


In [4]:
## Initialize embeddings manager
em=EmbeddingsManager(
    model_name='all-MiniLM-L6-v2',
    persist_directory='../data/vector_store'
)
print("Initialized Embeddings Manager")

Initialized Embeddings Manager


In [5]:
#Create collections
collection=em.create_collection(
    collection_name="policy_documents",
    reset=True
)
print("Created collection Policy Documents")

Created collection Policy Documents


In [6]:
em.add_documents(chunks=chunks)
print(f'Added {len(chunks)} documents to vector store')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Added 13 documents to vector store


In [7]:
#Get colllection stats
stats=em.get_collection_stats()
print("\n collcetion statics")
for key,value in stats.items():
    print(f"key:{key} Value:{value}")


 collcetion statics
key:total_documents Value:13
key:collection_name Value:policy_documents
key:persist_directory Value:../data/vector_store
key:sample_sections Value:['General', 'WAITING PERIODS AND IMMEDIATE COVERAGE', 'CLAIM LIMITS AND FINANCIAL CAPS', 'GEOGRAPHIC COVERAGE AND NETWORK HOSPITALS', 'SURGICAL COVERAGE AND BENEFITS', 'AGE']


In [9]:
# Cell 6: Test Search - Query 1
query1 = "knee surgery coverage"
results1 = em.search(query1, top_k=3)
print(f"\n🔍 Query: '{query1}'")
print(f"Found {len(results1['documents'])} results:\n")

for i, (doc, meta, dist) in enumerate(zip(
    results1['documents'],
    results1['metadatas'],
    results1['distances']
)):
    print(f"Result {i+1}:")
    print(f"  Section: {meta['section']}")
    print(f"  Distance: {dist:.4f}")
    print(f"  Text: {doc[:150]}...")
    print()


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


🔍 Query: 'knee surgery coverage'
Found 3 results:

Result 1:
  Section: SURGICAL COVERAGE AND BENEFITS
  Distance: 0.7193
  Text: SECTION 2: SURGICAL COVERAGE AND BENEFITS
─────────────────────────────────────────────────────────

2.1 ORTHOPEDIC PROCEDURES
Knee surgery, hip repla...

Result 2:
  Section: SURGICAL COVERAGE AND BENEFITS
  Distance: 1.1616
  Text: 2.2 CARDIAC PROCEDURES
Angioplasty, bypass surgery, and valve replacement covered at 90% of 
actual costs. Minimum policy duration: 6 months for elect...

Result 3:
  Section: EXCLUSIONS AND NON
  Distance: 1.2460
  Text: The following are NOT covered under this policy:
- Cosmetic or aesthetic procedures
- Experimental or unproven treatments
- Self-inflicted injuries
- ...



In [10]:
# Cell 7: Test Search - Query 2
query2 = "waiting period for new policy"
results2 = em.search(query2, top_k=3)

print(f"\n🔍 Query: '{query2}'")
print(f"Found {len(results2['documents'])} results:\n")

for i, (doc, meta, dist) in enumerate(zip(
    results2['documents'],
    results2['metadatas'],
    results2['distances']
)):
    print(f"Result {i+1}:")
    print(f"  Section: {meta['section']}")
    print(f"  Distance: {dist:.4f}")
    print(f"  Text: {doc[:150]}...")
    print()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


🔍 Query: 'waiting period for new policy'
Found 3 results:

Result 1:
  Section: WAITING PERIODS AND IMMEDIATE COVERAGE
  Distance: 0.6033
  Text: 1.1 STANDARD WAITING PERIOD
All policyholders must complete a mandatory 6-month waiting period 
before submitting claims for pre-existing medical cond...

Result 2:
  Section: General
  Distance: 0.7950
  Text: ═══════════════════════════════════════════════════════════
         HEALTHGUARD INSURANCE POLICY DOCUMENT
                    Policy Version 2.1
════...

Result 3:
  Section: WAITING PERIODS AND IMMEDIATE COVERAGE
  Distance: 0.9495
  Text: 1.2 IMMEDIATE COVERAGE
The following are covered immediately from policy start date with zero 
waiting period:
- Accidental injuries and emergency pro...



In [11]:
# Cell 8: Test Search - Query 3
query3 = "46 year old male knee surgery Pune"
results3 = em.search(query3, top_k=5)

print(f"\n🔍 Query: '{query3}'")
print(f"Found {len(results3['documents'])} results:\n")

for i, (doc, meta, dist) in enumerate(zip(
    results3['documents'],
    results3['metadatas'],
    results3['distances']
)):
    print(f"Result {i+1}:")
    print(f"  Section: {meta['section']}")
    print(f"  Distance: {dist:.4f}")
    print(f"  Text: {doc[:100]}...")
    print()


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


🔍 Query: '46 year old male knee surgery Pune'
Found 5 results:

Result 1:
  Section: AGE
  Distance: 1.3296
  Text: 4.1 STANDARD ELIGIBILITY (Ages 18-50)
Policyholders aged 18-50 years are eligible for all surgical p...

Result 2:
  Section: SURGICAL COVERAGE AND BENEFITS
  Distance: 1.3338
  Text: 2.2 CARDIAC PROCEDURES
Angioplasty, bypass surgery, and valve replacement covered at 90% of 
actual ...

Result 3:
  Section: SURGICAL COVERAGE AND BENEFITS
  Distance: 1.4348
  Text: SECTION 2: SURGICAL COVERAGE AND BENEFITS
─────────────────────────────────────────────────────────
...

Result 4:
  Section: AGE
  Distance: 1.5136
  Text: 4.3 SENIOR CITIZEN (Ages 65+)
Limited coverage available:
- Emergency procedures only
- No elective ...

Result 5:
  Section: EXCLUSIONS AND NON
  Distance: 1.6197
  Text: The following are NOT covered under this policy:
- Cosmetic or aesthetic procedures
- Experimental o...



In [12]:
# Cell 9: Analyze distance scores
import numpy as np

all_distances = results1['distances'] + results2['distances'] + results3['distances']
print("\n📊 Distance Score Analysis:")
print(f"  Min distance: {np.min(all_distances):.4f}")
print(f"  Max distance: {np.max(all_distances):.4f}")
print(f"  Mean distance: {np.mean(all_distances):.4f}")
print("\nNote: Lower distance = more relevant")


📊 Distance Score Analysis:
  Min distance: 0.6033
  Max distance: 1.6197
  Mean distance: 1.1551

Note: Lower distance = more relevant


In [13]:
# Cell 10: Test with section filtering (optional)
filtered_results = em.search(
    query="surgery",
    top_k=3,
    filter_metadata={"section": "SURGICAL COVERAGE AND BENEFITS"}
)

print(f"\n🔍 Filtered search (section='SURGICAL COVERAGE AND BENEFITS'):")
print(f"Found {len(filtered_results['documents'])} results")
for i, meta in enumerate(filtered_results['metadatas']):
    print(f"  {i+1}. Section: {meta['section']}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


🔍 Filtered search (section='SURGICAL COVERAGE AND BENEFITS'):
Found 2 results
  1. Section: SURGICAL COVERAGE AND BENEFITS
  2. Section: SURGICAL COVERAGE AND BENEFITS
